## Размещение баннеров
Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. 
Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас.
С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [32]:
import pandas as pd
check=pd.read_csv('C:/Users/gost_/Downloads/checkins.csv', delimiter='|', skipinitialspace=True)
check=check.dropna()
check=check[[3,4]]
check.head()

,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207
10,40.764462,-111.904565
11,33.448377,-112.074037


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [33]:
from sklearn.cluster import MeanShift
train=check.values[0:100000]
cl=MeanShift(bandwidth=0.1)
cl.fit(train)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.
При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации.

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:
* 33.751277, -118.188740 (Los Angeles)
* 25.867736, -80.324116 (Miami)
* 51.503016, -0.075479 (London)
* 52.378894, 4.885084 (Amsterdam)
* 39.366487, 117.036146 (Beijing)
* -33.868457, 151.205134 (Sydney)

In [34]:
labels=cl.labels_
centres=cl.cluster_centers_
print 'labels:\n', labels.shape, '\n\n', 'clasters:\n', centres.shape 

labels:
(100000L,) 

clasters:
(3230L, 2L)


Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [35]:
import numpy as np
def distance(x1, x2):
    return np.sqrt((x2[0]-x1[0])**2+(x2[1]-x1[1])**2)

In [52]:
coord_of=[[33.751277, -118.188740],
          [25.867736, -80.324116],
          [51.503016, -0.075479],
          [52.378894, 4.885084],
          [39.366487, 117.036146],
          [-33.868457, 151.205134]]
mins=[]
for c in centres:
    min=10000000000
    for of in coord_of:
        x=distance(c, of)
        if x<min:
            min=x
    mins.append((min, c))

In [53]:
mins.sort()
print mins[:20]

[(0.007834758163107856, array([ -33.86063043,  151.20477593])), (0.0093533161859922255, array([ 52.37296399,   4.89231722])), (0.022674066158385495, array([ 25.84567226, -80.3188906 ])), (0.050058294822787869, array([ 51.50299126,  -0.12553729])), (0.070847732427199731, array([  33.80987796, -118.14892381])), (0.13410903336184654, array([ 25.78581242, -80.21793804])), (0.154102829806012, array([ -34.00190615,  151.12806905])), (0.16740596425035326, array([ 25.70534972, -80.28342874])), (0.18887596060185083, array([ 26.01009825, -80.19999059])), (0.19222726571703244, array([ -33.9522629,  151.0321372])), (0.19577945647763628, array([  33.88832534, -118.04892817])), (0.21181053682436798, array([  33.87298601, -118.36209115])), (0.22223329073179071, array([  33.97257482, -118.16837067])), (0.24065314956287862, array([ 51.42676329,  -0.30373207])), (0.25150208676235281, array([ 52.388501  ,   4.63376547])), (0.26892862728711098, array([ 51.5741517,   0.1838708])), (0.27130075950667348, arr

In [54]:
print mins[0][1]

[ -33.86063043  151.20477593]
